In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://2e8125773246:4041
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1734135224899)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6862667a


In [10]:
import org.apache.spark.sql.{SparkSession, Dataset} 

val sparkSession = SparkSession.builder.appName("Juptyer").getOrCreate()

//TODO Illustrate how this fails if you change from Option[String] to String for referrer
case class Event (
    //Option is a way to handle NULL more gracefully
    user_id: Option[Integer],
    device_id: Option[Integer],
    referrer: Option[String],
    host: String,
    url: String,
    event_time: String
)


val dummyData = List(
    Event(user_id=Some(1), device_id=Some(2), referrer=Some("linkedin"), host="eczachly.com", url="/signup", event_time="2023-01-01"),
    Event(user_id=Some(3), device_id=Some(7), referrer=Some("twitter"), host="eczachly.com", url="/signup", event_time="2023-01-01")
)

//TODO Illustrate how this fails if you change from Option[Long] to Long
case class Device (
    device_id: Integer,
    browser_type: String,
    os_type: String,
    device_type: String
)

case class EventWithDeviceInfo (
   user_id: Integer,
    device_id: Integer,
    var browser_type: String,
    os_type: String,
    device_type: String,
    referrer: String,
    host: String,
    url: String,
    event_time: String
)

// When should you use each type?
import sparkSession.implicits._

// Applying this case class before hand is very powerful, enforces Nullability/non-nullability at runtime!
// Types are inferred in Scala but it is a good practice to include the type for all of your variables
val events: Dataset[Event] = sparkSession.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/events.csv")
                        .as[Event]

val devices: Dataset[Device] = sparkSession.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/devices.csv")
                        .as[Device]

devices.createOrReplaceTempView("devices")
events.createOrReplaceTempView("events")

// For simple transformations, you can see that these approaches are very similar. Dataset is winning slightly because of the quality enforcement
val filteredViaDataset = events.filter(event => event.user_id.isDefined && event.device_id.isDefined)
val filteredViaDataFrame = events.toDF().where($"user_id".isNotNull && $"device_id".isNotNull)
val filteredViaSparkSql = sparkSession.sql("SELECT * FROM events WHERE user_id IS NOT NULL AND device_id IS NOT NULL")

def toUpperCase(s: String): String = {
    return s.toUpperCase()
}

// This will fail if user_id is None
val combinedViaDatasets = filteredViaDataset
    .joinWith(devices, events("device_id") === devices("device_id"), "inner")
    .map{ case (event: Event, device: Device) => EventWithDeviceInfo(
                  // get is safe here because we are previously filtering out user_id nulls on filteredViaDataset
                  user_id=event.user_id.get,
                  device_id=device.device_id.get,
                  browser_type=device.browser_type,
                  os_type=device.os_type,
                  device_type=device.device_type,
                  referrer=event.referrer.getOrElse("Unknown"),
                  host=event.host,
                  url=event.url,
                  event_time=event.event_time
              ) }
    .map{ case (row: EventWithDeviceInfo) => {
        row.browser_type = toUpperCase(row.browser_type)
        row
    }}

val toUpperCaseUdf = udf(toUpperCase _)

// DataFrames give up some of the intellisense because you no longer have static typing
val combinedViaDataFrames = filteredViaDataFrame.as("e")
            //Make sure to use triple equals when using data frames
            .join(devices.as("d"), $"e.device_id" === $"d.device_id", "inner")
            .select(
              $"e.user_id",
              $"d.device_id",
              toUpperCaseUdf($"d.browser_type").as("browser_type"),
              $"d.os_type",
              $"d.device_type",
              $"e.referrer",
              $"e.host",
              $"e.url",
              $"e.event_time"
            )

//Creating temp views is a good strategy if you're leveraging SparkSQL
filteredViaSparkSql.createOrReplaceTempView("filtered_events")
val combinedViaSparkSQL = spark.sql(f"""
    SELECT 
        fe.user_id,
        d.device_id,
        d.browser_type,
        d.os_type,
        d.device_type,
        fe. referrer,
        fe.host,
        fe.url,
        fe.event_time
    FROM filtered_events fe 
    JOIN devices d ON fe.device_id = d.device_id
""")

combinedViaDatasets.take(5)

org.apache.spark.SparkException:  [INTERNAL_ERROR] The "head" action failed. You hit a bug in Spark or the Spark plugins you use. Please, report this bug to the corresponding communities or vendors, and provide the full stack trace.